**Requirements**
- Class roster, exported from Gradescope.
- Grades, exported from Gradescope (`.csv` files). e.g.
```
Multivitamin_1_scores.csv, Project_1_scores.csv, Project_1_Written_scores.csv
```
- Grading special cases spreadsheet, with the following columns
```
name, email, assignment, note, slip day, extension day, deduction
```


**Output**

- `grades.csv`, containing all student grades and compatible with the autograder.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
roster = pd.read_csv('/content/Data_101_Fall_2023_roster.csv', index_col='Email')
special = pd.read_csv('/content/Data 101 Fa23 Grading Special Cases.csv', index_col='email')

In [ ]:
def lateness_to_days(raw_time):
  if raw_time == 0:
    return 0
  hours = int(str(raw_time[:2]))
  if hours == 0:
    return 0
  else:
    return hours // 24.01 + 1

**Project Grades**

Four projects worth 52 points total. Each project is 13 points.

In [ ]:
proj_max_scores = {1: 27, 2: 67, 3: 27, 4: 28}

def proj_score(n):
  proj_score = pd.read_csv(f'/content/Project_{n}_scores.csv').fillna(0).set_index('Email')
  proj_score = proj_score[['Total Score', 'Lateness (H:M:S)']]
  proj_score[f'Proj {n} Lateness'] = proj_score['Lateness (H:M:S)'].apply(lateness_to_days)

  proj_written_score = pd.read_csv(f'/content/Project_{n}_Written_scores.csv').fillna(0).set_index('Email')
  proj_written_score = proj_written_score[['Total Score']]

  proj_score[f'Proj {n} Total Score'] = (proj_score['Total Score'] + proj_written_score['Total Score']) / proj_max_scores[n] * 13
  return proj_score

In [ ]:
for n in range(1, 5):
  df = proj_score(n)
  roster[f'Project {n} Lateness'] = df[f'Proj {n} Lateness']
  roster[f'Project {n} Score'] = df[f'Proj {n} Total Score']
  roster[f'Project {n} Auto-Allocated Slip Days'] = np.zeros(roster.shape[0])
  roster[f'Project {n} Traded-in Slip Days'] = np.zeros(roster.shape[0])
  roster[f'Project {n} Total Slip Days'] = np.zeros(roster.shape[0])
  roster[f'Project {n} Traded-in Slip Days'] = np.zeros(roster.shape[0])
  roster[f'Project {n} Late Penalty'] = np.zeros(roster.shape[0])
  roster[f'Project {n} Notes'] = np.full(roster.shape[0], 'N', dtype='str')


In [ ]:
# Record slip days and record late penalty for special cases
for n in range(1, 5):
  special_curr_assignment = special[special['assignment'] == f'proj {n}'].fillna(0)
  for email, row in special_curr_assignment.iterrows():
    roster.loc[email, f'Project {n} Notes'] = row['note']
    if row['deduction'] != 0:
      roster.loc[email, f'Project {n} Late Penalty'] = row['deduction']
      # roster.loc[email, f'Project {n} Score'] = (1 - row['deduction'] / 100) * roster.loc[email, f'Project {n} Score']
    if row['slip day'] != 0:
      roster.loc[email, f'Project {n} Traded-in Slip Days'] = row['slip day']

**Multivitamin Grades**

Five multivitamins worth 25 points total. Each multivitamin is 5 points.

In [ ]:
mv_max_scores = {1: 25, 2: 33, 3: 27, 4: 42, 5: 19}

def mv_score(n):
  mv_score = pd.read_csv(f'/content/Multivitamin_{n}_scores.csv').fillna(0).set_index('Email')
  mv_score =mv_score[['Total Score', 'Lateness (H:M:S)']]
  mv_score[f'Multivitamin {n} Lateness'] = mv_score['Lateness (H:M:S)'].apply(lateness_to_days)
  mv_score[f'Multivitamin {n} Total Score'] = mv_score['Total Score'] / mv_max_scores[n] * 5
  return mv_score

In [ ]:
for n in range(1, 6):
  df = mv_score(n)
  roster[f'Multivitamin {n} Lateness'] = df[f'Multivitamin {n} Lateness']
  roster[f'Multivitamin {n} Score'] = df[f'Multivitamin {n} Total Score']
  roster[f'Multivitamin {n} Auto-Allocated Slip Days'] = np.zeros(roster.shape[0])
  roster[f'Multivitamin {n} Traded-in Slip Days'] = np.zeros(roster.shape[0])
  roster[f'Project {n} Total Slip Days'] = np.zeros(roster.shape[0])
  roster[f'Multivitamin {n} Late Penalty'] = np.zeros(roster.shape[0])
  roster[f'Multivitamin {n} Notes'] = np.full(roster.shape[0], 'N', dtype='str')

In [ ]:
# Apply slip days and record penalty for special cases
for n in range(1, 6):
  special_curr_assignment = special[special['assignment'] == f'multi {n}'].fillna(0)
  for email, row in special_curr_assignment.iterrows():
    roster.loc[email, f'Multivitamin {n} Notes'] = row['note']
    if row['deduction'] != 0:
      roster.loc[email, f'Multivitamin {n} Late Penalty'] = row['deduction']
      # roster.loc[email, f'Multivitamin {n} Score'] = (1 - row['deduction'] / 100) * roster.loc[email, f'Multivitamin {n} Score']
    if row['slip day'] != 0:
      roster.loc[email, f'Multivitamin {n} Traded-in Slip Days'] = row['slip day']

**Lecture Attendance Grades**

Lecture check-ins worth 8 points total. Drop three. Calculation M / (T - 3). Each worth 8 / 24 points. Capped at 8 points.


In [ ]:
def lec_score():
  lec_score = pd.read_csv(f'/content/Lecture_Attendance_scores.csv', index_col='Email').fillna(0)
  lec_score =lec_score[['Total Score']]
  lec_score['Lecture Attendance Total Score'] = np.minimum(lec_score['Total Score'] / 24 * 8, 8)
  return lec_score

In [ ]:
 df = lec_score()
 roster[f'Lecture Attendance Score'] = df['Lecture Attendance Total Score']

**Slip Days**

Calculate Total Late Days & Allocable Slip Days

In [ ]:
late, slip = 0, 0
for n in range(1, 5):
  late += roster[f'Project {n} Lateness']
  slip += roster[f'Project {n} Traded-in Slip Days']

for n in range(1, 6):
  late += roster[f'Multivitamin {n} Lateness']
  slip += roster[f'Multivitamin {n} Traded-in Slip Days']

roster['Total Lateness'] = late
roster['Total Slip Days Remaining'] = 9 - slip

Determine how to apply slip days for students who have more late days than remaining slip days

In [ ]:
to_be_updated = list(roster[roster['Total Lateness'] > roster['Total Slip Days Remaining']].index)
to_be_updated

In [ ]:
lateness_items = []
for i in range(1, 5):
  lateness_items.append(f'Project {i}')
for i in range(1, 6):
  lateness_items.append(f'Multivitamin {i}')
lateness_items

In [ ]:
for email in roster.index:
  if email not in to_be_updated:
    for item in lateness_items:
      slip_day_curr = roster.loc[email, item + ' Lateness']
      if slip_day_curr > 0:
        slip_day_curr = int(slip_day_curr)
        roster.loc[email, item + ' Auto-Allocated Slip Days'] = slip_day_curr
        roster.loc[email, 'Total Slip Days Remaining'] -= slip_day_curr

In [ ]:
to_process = {}
for email in to_be_updated:
  student_late_items = []
  for item in lateness_items:
    slip_day_curr = roster.loc[email, item + ' Lateness']
    if slip_day_curr > 0:
      slip_day_curr = int(slip_day_curr)
      student_late_items.append(item)
  student_late_items.sort(key=lambda item: roster.loc[email, item + ' Score'], reverse=True)
  print(email, student_late_items)
  to_process[email] = student_late_items

In [ ]:
for email in to_process:
  print(f'Start processing {email}:')
  items = to_process[email]
  print(f'Late assignments: {items}')
  items_to_remove = []
  for item in items:
    lateness_curr = roster.loc[email, item + ' Lateness']
    slip_days_remaining = roster.loc[email, 'Total Slip Days Remaining']
    if slip_days_remaining >= lateness_curr:
      roster.loc[email, item + ' Auto-Allocated Slip Days'] += lateness_curr
      roster.loc[email, 'Total Slip Days Remaining'] -= lateness_curr
      items_to_remove.append(item)
      print(f'No penalty on {item}')


    else:
      while slip_days_remaining > 0:
        roster.loc[email, item + ' Auto-Allocated Slip Days'] += 1
        roster.loc[email, 'Total Slip Days Remaining'] -= 1
        slip_days_remaining -= 1

  for item in items_to_remove:
    items.remove(item)
  print(f'Remaining: {items}')
  for item in items:
    penalty_days = roster.loc[email, item + ' Lateness'] - roster.loc[email, item + ' Auto-Allocated Slip Days']
    roster.loc[email, item + ' Late Penalty'] = penalty_days * 15.0

Make a final pass: apply penalty (if any) to grades

In [ ]:
assig_items = lateness_items.copy()
for item in assig_items:
  roster.loc[:, f'{item} Score'] = roster.loc[:, f'{item} Score'] * (100.0 - roster.loc[:, f'{item} Late Penalty']) / 100.0


**Final Exam Grades**

Add Final Exam Grades: Final Exam is out of 125 points, accounts for 15% of overall grade.

In [ ]:
final = pd.read_csv('Final_Exam_scores.csv', index_col='Email')
for email in roster.index:
  roster.loc[email, 'Final Exam Score'] = final.loc[email, 'Total Score'] / 125 * 15

**Total Score**

Calculate total score.

In [ ]:
roster = roster.fillna(0)

In [ ]:
total = 0
for n in range(1, 5):
  total += roster[f'Project {n} Score']

roster['Final Project Score'] = total

total = 0
for n in range(1, 6):
  total += roster[f'Multivitamin {n} Score']

roster['Final Multivitamin Score'] = total

total = 0
for n in range(1, 5):
  total += roster[f'Project {n} Score']

for n in range(1, 6):
  total += roster[f'Multivitamin {n} Score']

total += roster['Lecture Attendance Score']
total += roster['Final Exam Score']

roster['Final Total Score'] = total

In [ ]:
roster = roster.reset_index()
roster.head()

In [ ]:
roster.to_csv('grades.csv')